In [3]:
from sagemaker import image_uris, Model
import sagemaker
import sys
sys.path.append('../')
from utils.config import boto_sess,role,sagemaker_session,DEFAULT_REGION,SUPPORTED_MODELS_FILE,DEFAULT_TEMPLATE_FILE,default_bucket,VLLM_IMAGE,MODEL_ARTIFACT


/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


environ({'USER': 'ubuntu', 'SSH_CLIENT': '111.198.223.106 58543 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'HOME': '/home/ubuntu', 'SSL_CERT_FILE': '/usr/lib/ssl/cert.pem', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'LOGNAME': 'ubuntu', '_': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin/python', 'XDG_SESSION_CLASS': 'user', 'VSCODE_CLI_REQUIRE_TOKEN': '08749abe-85df-47fe-a1e0-ea3c34d7468b', 'XDG_SESSION_ID': '2809', 'PATH': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/workspace/llm_model_hub/miniconda3/condabin:/home/ubuntu/.vscode-server/cli/servers/Stable-fee1edb8d6d72a0ddff41e5f71a671c23ed924b9/server/bin/remote-cli:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/ubuntu/.vscode-server', 'XDG_RUNTIME_DIR': '/run/user/1000', 'SSL_CERT_DIR': '/usr/lib/ssl/certs', 'LANG': 'C.UTF-8', 'SHELL': '/bin/bash', 'PWD': '/home/ubuntu', 'SSH_CONN

In [4]:
lmi_image_uri = image_uris.retrieve(framework="djl-lmi", version="0.29.0", region=DEFAULT_REGION)

Defaulting to the only supported framework/algorithm version: 0.28.0. Ignoring framework/algorithm version: 0.29.0.


In [8]:
import os
model_path = "s3://sagemaker-us-east-1-434444145045/Meta-Llama-3-1-8B-Instruct/33dba50f38f34e788ce0abea2ba31743/finetuned_model_merged/"

In [10]:
endpoint_name = sagemaker.utils.name_from_base('llama3.1-8b-mod').replace('.','-').replace('_','-')
instance_type = "ml.g5.2xlarge"
env={
    "HF_MODEL_ID": model_path,
    "OPTION_ROLLING_BATCH":  "lmi-dist",
    "TENSOR_PARALLEL_DEGREE": "max",
    "OPTION_TRUST_REMOTE_CODE": "true",
     "HUGGING_FACE_HUB_TOKEN":os.environ.get('HUGGING_FACE_HUB_TOKEN'),
}

# Create the SageMaker Model object. In this example we let LMI configure the deployment settings based on the model architecture  
model = Model(
        image_uri=lmi_image_uri,
        role=role,
        name=endpoint_name,
        sagemaker_session=sagemaker_session,
        env=env,
)

model.deploy(
    instance_type= instance_type,
    initial_instance_count=1,
    endpoint_name=endpoint_name,
    wait=True,
    accept_eula=True,
    container_startup_health_check_timeout=900
)

----------------!

In [1]:
import boto3,json
runtime = boto3.client('runtime.sagemaker',region_name='us-east-1')

In [5]:
endpoint_name = 'Meta-Llama-3-1-8B-Instruct-2024-09-04-10-26-26-004'
payload = {
    # "model": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "model":"meta-llama/Meta-Llama-3.1-8B-Instruct",
    "messages": [
    {
        "role": "user",
        "content": "hello, who is the main charater of movie Titanic"
    }
    ],
    "max_tokens": 1024,
    "stream": False
}
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])

The two main characters of the movie Titanic are:

1. **Jack Dawson** (played by Leonardo DiCaprio): A poor artist who wins his ticket to board the ship in a poker game. He is a free spirit who falls in love with Rose.
2. **Rose DeWitt Bukater** (played by Kate Winslet): A high-society woman engaged to marry a wealthy man, Cal Hockley. She falls in love with Jack on the ship.

However, if you're asking about the main character, I'd say it's **Rose DeWitt Bukater**. The movie starts with an elderly Rose telling the story of her time on the Titanic to her granddaughter, and the story revolves around her experiences and emotions.

But, if you consider the character that drives the plot forward, it's **Jack Dawson**. His character represents the free-spirited and romantic side of the story, and his relationship with Rose is the central theme of the movie.

It's worth noting that James Cameron, the director of the movie, has stated that he considers Jack and Rose to be co-main characters, r